In [1]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_datasets

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [3]:
train_data, val_data, test_data = tfds.load(name="imdb_reviews",
                                           split = ("train","test[:40%]","test[40%:]"),
                                           as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GAZXTR_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GAZXTR_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GAZXTR_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
print(train_data)

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [5]:
text, label = next(iter(train_data.batch(5)))

In [6]:
label

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 1, 1])>

In [7]:
text

<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [8]:
hub_layer = hub.KerasLayer("https://kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1", output_shape=[20],
                           input_shape=[], dtype=tf.string)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 4)                 68        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 400429 (1.53 MB)
Trainable params: 409 (1.60 KB)
Non-trainable params: 400020 (1.53 MB)
_________________________________________________________________


In [9]:
model.compile(optimizer = 'adam',loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics = ['accuracy'])

In [10]:
history = model.fit(train_data.shuffle(10000).batch(100), epochs = 25, validation_data = val_data.batch(100),verbose = 1)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


250/250 [==============================] - 6s 18ms/step - loss: 0.6904 - accuracy: 0.5469 - val_loss: 0.6548 - val_accuracy: 0.6373
Epoch 2/25
250/250 [==============================] - 3s 13ms/step - loss: 0.6157 - accuracy: 0.6721 - val_loss: 0.5899 - val_accuracy: 0.6859
Epoch 3/25
250/250 [==============================] - 3s 13ms/step - loss: 0.5683 - accuracy: 0.7107 - val_loss: 0.5685 - val_accuracy: 0.7067
Epoch 4/25
250/250 [==============================] - 4s 17ms/step - loss: 0.5599 - accuracy: 0.7167 - val_loss: 0.5647 - val_accuracy: 0.7076
Epoch 5/25
250/250 [==============================] - 4s 13ms/step - loss: 0.5550 - accuracy: 0.7192 - val_loss: 0.5613 - val_accuracy: 0.7120
Epoch 6/25
250/250 [==============================] - 3s 13ms/step - loss: 0.5522 - accuracy: 0.7211 - val_loss: 0.5601 - val_accuracy: 0.7139
Epoch 7/25
250/250 [==============================] - 5s 18ms/step - loss: 0.5510 - accuracy: 0.7206 - val_loss: 0.5568 - val_accuracy: 0.7127
Epoch 8/25

In [11]:
results = model.evaluate(test_data.batch(100), verbose = 2)

150/150 - 2s - loss: 0.5464 - accuracy: 0.7222 - 2s/epoch - 12ms/step
